In [0]:
from pyspark.sql.functions import current_date, col, datediff, max as spark_max


In [0]:
df = spark.sql(f"SELECT * FROM dev.job_prospects.job_1900_silver_inferred")
df.display()

In [0]:
max_timestamp = df.agg(spark_max("timestamp").alias("max_ts")).collect()[0]["max_ts"]

df_last_7_days = df.filter(
    (col("timestamp") == max_timestamp)
    & (datediff(current_date(), col("posted_at")) <= 7)
)

df_last_30_days = df.filter(
    (col("timestamp") == max_timestamp)
    & (datediff(current_date(), col("posted_at")) <= 30)
)

In [0]:
df_top_10_job_last_7_days = df_last_7_days.groupBy("job_field").count().orderBy(col("count").desc()).limit(10)
df_top_10_job_last_30_days = df_last_30_days.groupBy("job_field").count().orderBy(col("count").desc()).limit(10)

In [0]:
df_top_10_job_last_7_days.write.mode("overwrite").saveAsTable("dev.job_prospects_gold.fct_top_10_last_week")